In [ ]:
import pandas as pd
import re
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, roc_curve, auc
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/HateSpeechDataset.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)
df.head()

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"


In [ ]:
train = df.drop(['Content_int'], axis = 1)

In [ ]:
import nltk
import string
import re

# Download the 'stopwords' resource
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)



#Removing the stopwords from text
def split_into_words(text):
    # split into words by white space
    words = text.split()
    return words

def to_lower_case(words):
    # convert to lower case
    words = [word.lower() for word in words]
    return words

def remove_punctuation(words):
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    stripped = [re_punc.sub('', w) for w in words]
    return stripped

def keep_alphabetic(words):
    # remove remaining tokens that are not alphabetic
    words = [word for word in words if word.isalpha()]
    return words

def remove_stopwords(words):
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

def to_sentence(words):
    # join words to a sentence
    return ' '.join(words)
def tweet(words):
    tweet_tokenizer = nltk.tokenize.TweetTokenizer(strip_handles=True,reduce_len=True)
    tweet = tweet_tokenizer.tokenize(words)
    return tweet

#Removing the noisy text
def denoise_text(text):
    words = split_into_words(text)
    words = to_lower_case(words)
    words = remove_punctuation(words)
    words = keep_alphabetic(words)
    words = remove_stopwords(words)
    return to_sentence(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train['Content']=train['Content'].apply(denoise_text)

In [ ]:
train

,Content,Label
0,denial normal con asked comment tragedies emot...,1
1,able tweet insufferable bullshit proves trump ...,1
2,retarded cute single life,1
3,thought real badass mongol style declaration w...,1
4,afro american basho,1
...,...,...
440901,crash another movie left field ask star wars e...,0
440902,debate first starting edit war,0
440903,removal reverted removal section believe misgu...,0
440904,unblocked eddie discussed user talkable accusi...,0


In [ ]:
X = train.Content
Y = train.Label
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [ ]:
from keras import utils
max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = utils.pad_sequences(sequences,maxlen=max_len)

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation

def CNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(layer)
    #make this to 256
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Flatten()(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model


In [ ]:
model = CNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 50)           50000     
                                                                 
 conv1d (Conv1D)             (None, 200, 64)           9664      
                                                                 
 max_pooling1d (MaxPooling1  (None, 100, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 100, 128)          24704     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 50, 128)           0         
 g1D)                                                        

In [ ]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=2,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])




Epoch 1/2
2343/2343 [==============================] - 427s 182ms/step - loss: 0.3606 - accuracy: 0.8382 - val_loss: 0.3471 - val_accuracy: 0.8452
Epoch 2/2
2343/2343 [==============================] - 426s 182ms/step - loss: 0.3401 - accuracy: 0.8488 - val_loss: 0.3518 - val_accuracy: 0.8477


In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.2)

# Make predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Generate classification report
report = classification_report(y_test_classes, y_pred_classes, target_names=[str(i) for i in range(10)])
print(report)


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
375/375 [==============================] - 24s 62ms/step - loss: 0.2578 - accuracy: 0.9264 - val_loss: 0.1029 - val_accuracy: 0.9718
Epoch 2/5
375/375 [==============================] - 24s 63ms/step - loss: 0.0783 - accuracy: 0.9773 - val_loss: 0.0681 - val_accuracy: 0.9807
Epoch 3/5
375/375 [==============================] - 24s 63ms/step - loss: 0.0522 - accuracy: 0.9844 - val_loss: 0.0648 - val_accuracy: 0.9819
Epoch 4/5
375/375 [==============================] - 24s 65ms/step - loss: 0.0387 - accuracy: 0.9885 - val_loss: 0.0539 - val_accuracy: 0.9839
Epoch 5/5
313/313 [==============================] - 2s 5ms/step
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.99      1.00      0.99      1135
           2       0.98      0.99      0.98      1032
           3       0.98      0.99      0.99      1010
           4       0.98    